In [1]:
#Declaración de paquetes
import pandas as pd
import sys
import gc
sys.path.append('/DataDF/Analytics/DDNA_package')
from SqlServerAdapter import SqlServerAdapter

conn = SqlServerAdapter('sqlServer_datahub_dev') 
#Configuraciones
pd.options.mode.chained_assignment = None

#input
df = pd.read_csv('/DataDF/Analytics/JuanSalinas/rcc202204.ope', sep=r';;+',header=None, names=['Field_1'],
                 encoding = "ISO-8859-1",dtype={'Field_1': 'str'},engine='python')
#output
schema = 'D_DDNA_LANDING'
tb_deuda = 'SDeuda'
tb_sbscliente = 'SbsGral'

len_particion = 1000000

Conexión encriptada exitosa
dbConfig.json was successful


In [3]:
#Generando los dataframe de Clientes y Deudas
df['value'] = df['Field_1'].str[0:1]
df_SBSCliente = df[df['value']=='1'].copy()
df_SBSCliente = df_SBSCliente.reset_index(drop=True)

#df_SBSDeuda = df[df['value']=='2'].copy()
#df_SBSDeuda = df_SBSDeuda.reset_index(drop=True)
display(df_SBSCliente)

,Field_1,value
0,10038518267|20220430| ||1|29409477|1| |002|000...,1
1,10029280002|20220430| ||1|06146421|1| |001|000...,1
2,10016584398|20220430| ||1|07699936|1| |001|000...,1
3,10050957942|20220430| ||1|00478547|1| |001|000...,1
4,10015096683|20220430| ||1|06210773|1| |001|000...,1
...,...,...
11823004,10000497088|20220430|3|20101004610|||2| |001|0...,1
11823005,10139552431|20220430|3|20539251971|||2| |001|1...,1
11823006,10107420231|20220430| ||1|44238764|1| |001|100...,1
11823007,10081728950|20220430| ||1|45419390|1| |003|100...,1


In [12]:


conn.query("""TRUNCATE TABLE """ + schema + "." + tb_deuda, commit=True)
for i in range(0, len(df_SBSDeuda), len_particion):
    df_particion = df_SBSDeuda.loc[i:i+len_particion-1, :].copy()
    print("antes: ",df_particion.count())
    print(i,"segundo indice: ",i+len_particion-1)
    df_particion['CodigoSBS'] = df_particion['Field_1'].str[1:11]
    df_particion['CodigoEmpresa'] = df_particion['Field_1'].str[11:16]
    df_particion['TipoCredito'] = df_particion['Field_1'].str[16:18]
    df_particion['Nivel2'] = df_particion['Field_1'].str[18:20]
    df_particion['Moneda'] = df_particion['Field_1'].str[20:21]
    df_particion['SubCodigoCuenta'] = df_particion['Field_1'].str[21:32]
    df_particion['Condicion'] = df_particion['Field_1'].str[32:38]
    df_particion['ValorSaldo'] = df_particion['Field_1'].str[38:42]
    df_particion['ClasificacionDeuda'] = df_particion['Field_1'].str[42:43]
    
    df_particion['CodigoCuenta'] = df_particion['Nivel2'] + df_particion['Moneda'] + df_particion['SubCodigoCuenta']

    #Converción de formato
    df_particion[['CodigoSBS']] = df_particion[['CodigoSBS']].astype(float)
    df_particion[['TipoCredito']] = df_particion[['TipoCredito']].astype(int)
    df_particion[['CodigoEmpresa']] = df_particion[['CodigoEmpresa']].astype(int)
    df_particion[['Condicion']] = df_particion[['Condicion']].astype(int)
    df_particion[['ClasificacionDeuda']] = df_particion[['ClasificacionDeuda']].astype(int)
    df_particion[['Moneda']] = df_particion[['Moneda']].astype(int)
    df_particion[['Nivel2']] = df_particion[['Nivel2']].astype(int)

    #Renombrar columnas 
    colum_deuda= { 'CodigoSBS':'Cod_SBS','CodigoEmpresa':'Cod_Emp','TipoCredito':'Tip_Credit','CodigoCuenta':'Cod_Cuenta',
                        'Condicion':'Condicion','ValorSaldo':'Val_Saldo','ClasificacionDeuda':'Clasif_Deu'}
    df_particion = df_particion.rename(columns=colum_deuda)

    df_particion = df_particion.drop(['Field_1'], axis=1)
    df_particion = df_particion.drop(['SubCodigoCuenta'], axis=1)
    df_particion = df_particion.drop(['value'], axis=1)

    print(">>Insertando Deudas", tb_deuda)
    print("despues: ",df_particion.count())
    conn.write(df_particion, schema, tb_deuda, parallel=True)

starting query
The connection was successful
['0', '0', '0', '0', '0', 'SUCCESSFUL']
TRUNCATE TABLE D_DDNA_LANDING.SDeuda
antes:  Field_1    1000000
value      1000000
dtype: int64
0 segundo indice:  999999
>>Insertando Deudas SDeuda
despues:  Cod_SBS       1000000
Cod_Emp       1000000
Tip_Credit    1000000
Nivel2        1000000
Moneda        1000000
Condicion     1000000
Val_Saldo     1000000
Clasif_Deu    1000000
Cod_Cuenta    1000000
dtype: int64
Dataframe to insert : (1000000, 9)
parallel=True 
The connection was successful
The connection was successful
The connection was successful
Iniciando la inserción:
Shape of dataframe to insert : (100000, 9)
The connection was successful
Shape of dataframe to insert : (100000, 9)
Iniciando la inserción:
Shape of dataframe to insert : (100000, 9)
Iniciando la inserción:
Shape of dataframe to insert : (100000, 9)
Iniciando la inserción:
Terminando método write
Desconectado
The connection was successful
Shape of dataframe to insert : (100000, 

In [ ]:
df_SBSDeuda['CodigoSBS'] = df_SBSDeuda['Field_1'].str[1:11]
df_SBSDeuda['CodigoEmpresa'] = df_SBSDeuda['Field_1'].str[11:16]
df_SBSDeuda['TipoCredito'] = df_SBSDeuda['Field_1'].str[16:18]
df_SBSDeuda['Nivel2'] = df_SBSDeuda['Field_1'].str[18:20]
df_SBSDeuda['Moneda'] = df_SBSDeuda['Field_1'].str[20:21]
df_SBSDeuda['SubCodigoCuenta'] = df_SBSDeuda['Field_1'].str[21:32]
df_SBSDeuda['Condicion'] = df_SBSDeuda['Field_1'].str[32:38]
df_SBSDeuda['ValorSaldo'] = df_SBSDeuda['Field_1'].str[38:42]
df_SBSDeuda['ClasificacionDeuda'] = df_SBSDeuda['Field_1'].str[42:43]
#Transformación
df_SBSDeuda['CodigoCuenta'] = df_SBSDeuda['Nivel2'] + df_SBSDeuda['Moneda'] + df_SBSDeuda['SubCodigoCuenta']

#Converción de formato
df_SBSDeuda[['CodigoSBS']] = df_SBSDeuda[['CodigoSBS']].astype(float)
df_SBSDeuda[['TipoCredito']] = df_SBSDeuda[['TipoCredito']].astype(int)
df_SBSDeuda[['CodigoEmpresa']] = df_SBSDeuda[['CodigoEmpresa']].astype(int)
df_SBSDeuda[['Condicion']] = df_SBSDeuda[['Condicion']].astype(int)
df_SBSDeuda[['ClasificacionDeuda']] = df_SBSDeuda[['ClasificacionDeuda']].astype(int)
df_SBSDeuda[['Moneda']] = df_SBSDeuda[['Moneda']].astype(int)
df_SBSDeuda[['Nivel2']] = df_SBSDeuda[['Nivel2']].astype(int)

#Renombrar columnas 
colum_deuda= { 'CodigoSBS':'Cod_SBS','CodigoEmpresa':'Cod_Emp','TipoCredito':'Tip_Credit','CodigoCuenta':'Cod_Cuenta',
                    'Condicion':'Condicion','ValorSaldo':'Val_Saldo','ClasificacionDeuda':'Clasif_Deu'}
df_SBSDeuda = df_SBSDeuda.rename(columns=colum_deuda)
display(df_SBSDeuda)
df_SBSDeuda = df_SBSDeuda.drop(['Field_1'], axis=1)
df_SBSDeuda = df_SBSDeuda.drop(['SubCodigoCuenta'], axis=1)
df_SBSDeuda = df_SBSDeuda.drop(['value'], axis=1)

display(df_SBSDeuda)

In [ ]:
df_SBSCliente.to_csv('SBSCliente2.csv', index = False)
df_SBSDeuda.to_csv('SBSDeuda2.csv', index = False)

In [131]:
print(">>Insertando Deudas", tb_deuda)
conn.write(df_SBSDeuda, schema, tb_deuda, parallel=True)

starting query
The connection was successful
['0', '0', '0', '0', '0', 'SUCCESSFUL']
TRUNCATE TABLE D_DDNA_LANDING.SDeuda
>>Insertando Deudas SDeuda
Dataframe to insert : (861, 9)
parallel=True 
The connection was successful
The connection was successful
Shape of dataframe to insert : (215, 9)
The connection was successful
The connection was successful
Shape of dataframe to insert : (215, 9)
Iniciando la inserción:
Iniciando la inserción:
Shape of dataframe to insert : (215, 9)
Shape of dataframe to insert : (216, 9)
Iniciando la inserción:
Iniciando la inserción:
Terminando método write
Desconectado
Terminando método write
Terminando método write
Desconectado
Desconectado
Terminando método write
Desconectado


In [7]:
#Lógica de SBSCliente

# Se elimina los registros de la tabla
conn.query("""TRUNCATE TABLE """ + schema + "." + tb_sbscliente, commit=True)

#Creando los columnas con el delimitador |
#df_SBSCliente = df_SBSCliente["Field_1"].str.split('|', expand=True)

#Generando el diccionario para reemplazar el nombre de columnas
columnas_cliente = { 0:'SBSCodigoCliente',1:'SBSFechaReporte',2:'SBSTipoDocumentoT',3:'SBSRucCliente',
                    4:'SBSTipoDocumento',5:'SBSNumeroDocumento',6:'SBSTipoPer',7:'SBSTipoEmpresa',
                    8:'SBSNumeroEntidad',9:'SBSSalNor',10:'SBSSalCPP',11:'SBSSalDEF',12:'SBSSalDUD',
                    13:'SBSSalAPER',14:'SBSAPEPAT',15:'SBSAPEMAT',16:'SBSAPECAS',17:'SBSNOMCLI',18:'SBSNOMCLI2'}

df_SBSCliente = df_SBSCliente.rename(columns=columnas_cliente)
#df_SBSCliente.columns
for i in range(0, len(df_SBSCliente), len_particion):
    df_particion_2 = df_SBSCliente.loc[i:i+len_particion-1, :].copy()
    #Se modifica el tipo de dato de object a float o int
    df_particion_2[['SBSSalNor', 'SBSSalCPP','SBSSalDEF','SBSSalDUD','SBSSalAPER']] = df_SBSCliente[
        ['SBSSalNor', 'SBSSalCPP','SBSSalDEF','SBSSalDUD','SBSSalAPER']].astype(float)
    df_particion_2[['SBSNumeroEntidad']] = df_particion_2[['SBSNumeroEntidad']].astype(int)

    #Se generan calculos en base a la lógica de negocio
    df_particion_2['SBSSalNor'] = df_particion_2['SBSSalNor'] / 100
    df_particion_2['SBSSalCPP'] = df_particion_2['SBSSalCPP'] / 100
    df_particion_2['SBSSalDEF'] = df_particion_2['SBSSalDEF'] / 100
    df_particion_2['SBSSalDUD'] = df_particion_2['SBSSalDUD'] / 100
    df_particion_2['SBSSalAPER'] = df_particion_2['SBSSalAPER'] / 100
    df_particion_2['Periodo'] = df_particion_2['SBSFechaReporte'].str[:4]
    df_particion_2['CodigoSBS'] = df_particion_2['SBSCodigoCliente'].str[1:]
    df_particion_2 = df_particion_2.replace(to_replace =["#"], value ="Ñ") 


    #Renombrar columnas de acuerdo a como se encuentran en la base de datos
    column_sbscliente= {'CodigoSBS':'Cod_SBS',
                  'SBSFechaReporte':'Fec_Rep',
                  'SBSTipoDocumentoT':'Tip_Doc_Tr',
                  'SBSRucCliente':'Cod_Doc_Tr',
                  'SBSTipoDocumento':'Tip_Doc_I',
                  'SBSNumeroDocumento':'Cod_Doc_I',
                  'SBSTipoPer':'Tip_Per',
                  'SBSTipoEmpresa':'Tip_Emp',
                  'SBSNumeroEntidad':'Can_Ents',
                  'SBSSalNor':'Calif_0',
                  'SBSSalCPP':'Calif_1',
                  'SBSSalDEF':'Calif_2',
                  'SBSSalDUD':'Calif_3',
                  'SBSSalAPER':'Calif_4',
                  'SBSAPEPAT':'Nom_Deu',
                  'SBSAPEMAT':'Amaterno',
                  'SBSAPECAS':'Acasada',
                  'SBSNOMCLI':'Nombre1',
                  'SBSNOMCLI2':'Nombre2',
                 }
    df_particion_2 = df_particion_2.rename(columns=column_sbscliente)

    # Se eliminan columnas no necesarias
    df_particion_2 = df_particion_2.drop(['SBSCodigoCliente'], axis=1)
    df_particion_2 = df_particion_2.drop(['Periodo'], axis=1)

    # Se inserta los nuevos registros en la tabla
    print(">>Insertando Deudas", tb_sbscliente)
    conn.write(df_particion_2, schema, tb_sbscliente, parallel=True)

starting query
The connection was successful
['0', '0', '0', '0', '0', 'SUCCESSFUL']
TRUNCATE TABLE D_DDNA_LANDING.SbsGral
>>Insertando Deudas SbsGral
Dataframe to insert : (1000000, 19)
parallel=True 
The connection was successful
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
The connection was successful
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
The connection was successful
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
The connection was successful
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
Terminando método write
Desconectado
The connection was successful
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
Terminando método write
Desconectado
Terminando método write
Desconectado
The connection was successful
Terminando método write
Desconectado
Shape of dataframe to insert : (100000, 19)
Iniciando la inserción:
The connection was successful
Shape of dataframe to insert : (